In [ ]:
import os
from m_libri_generator import get_file_dict_libri
import numpy as np
import torch.nn.functional as F
import torch
import random
import matplotlib.pyplot as plt
import tuneThreshold
import csv

In [ ]:
# EMB_PATH = "../libri_emb/base/"
# EMB_PATH = "../libri_emb/trimmed/"
EMB_PATH = "../libri_emb/test/"

In [ ]:
npy_list = os.listdir(EMB_PATH)
npy_list = [i for i in npy_list if str.find(i, "npy") != -1]

spk_emb_dict = {}
for emb in npy_list:
    emb_spk = int(emb[3:-4])
    emb_mat = np.load(EMB_PATH+emb)
    
    emb_mat = torch.tensor(emb_mat)
    spk_emb_dict[emb_spk] = emb_mat

spk_list = list(spk_emb_dict.keys())

In [ ]:
spk_npy_dict = {}
for spk in spk_list:
    structure_dict = {}
    csv_file = open(EMB_PATH+'op_{}.csv'.format(spk), 'r')
    csv_reader = csv.reader(csv_file)
    
    for row in csv_reader:
        wav_idx = int(row[0])
        wav_name = row[1]
        wav_cat = int(row[1].split('-')[1])
        
        cat_list = structure_dict.get(wav_cat, [])
        cat_list.append(wav_idx)
        structure_dict[wav_cat] = cat_list
    
    spk_npy_dict[spk] = structure_dict
    csv_file.close()

In [ ]:
# 단일 발화 EER - original

def eer_original(iter=10, diff_cat_flag=False):
    d0_same = []
    d0_diff = []
    
    for i in range(iter):        
        for spk in spk_list:
            
            # same spk
            emb_mat = spk_emb_dict[spk]
            
            # 다른 카테고리 flag
            if diff_cat_flag:
                npy_dict = spk_npy_dict[spk]
                cats = list(npy_dict.keys())
                
                # 화자의 카테고리 수가 1인 경우, 다른 카테고리에서 두개 뽑는게 불가능하므로 넘김.
                if len(cats) == 1:
                    continue
                
                selected_cats = random.sample(cats, 2)
                cat_list1 = npy_dict[selected_cats[0]]
                cat_list2 = npy_dict[selected_cats[1]]
                
                indices = []
                indices.append(random.choice(cat_list1))
                indices.append(random.choice(cat_list2))
            
            else:
                indices = random.sample(range(emb_mat.shape[0]), 2)
            
            emb1 = emb_mat[indices[0]]
            emb2 = emb_mat[indices[1]]
            
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)
            d0_same.append(torch.cdist(emb1, emb2, p=2).mean().item())
            
            # diff spk
            indices = random.sample(spk_list, 2)
            emb_mat1 = spk_emb_dict[indices[0]]
            emb_mat2 = spk_emb_dict[indices[1]]
            emb1 = random.choice(emb_mat1)
            emb2 = random.choice(emb_mat2)
            
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)
            d0_diff.append(torch.cdist(emb1, emb2, p=2).mean().item())

    fig, ax = plt.subplots()
    plt.boxplot([d0_same, d0_diff], positions=[1,0])
    ax.set_xlabel('Label')
    ax.set_ylabel('Distance')
    plt.show()

    d0 = d0_same + d0_diff
    s0 = [-1*i for i in d0]
    label_same = np.ones(len(d0_same))
    label_diff = np.zeros(len(d0_diff))
    label = list(np.concatenate([label_same, label_diff]))
    (tt0, eer0, fpr0, fnr0) = tuneThreshold.tuneThresholdfromScore(s0, label, [1, 0.1])
    
    return eer0

In [ ]:
def eer_single_normalize(iter=10, flag=0, diff_cat_flag=False):
    '''
    flag(0): normalize before frame-by average
    flag(1): normalize after frame-by average
    '''
    assert flag == 0 or flag == 1
    
    d1_same = []
    d1_diff = []
    
    for i in range(iter):
        for spk in spk_list:

            # same spk
            emb_mat = spk_emb_dict[spk] # n_utter, 10, 512
            
            # 다른 카테고리 flag
            if diff_cat_flag:
                npy_dict = spk_npy_dict[spk]
                cats = list(npy_dict.keys())
                
                # 화자의 카테고리 수가 1인 경우, 다른 카테고리에서 두개 뽑는게 불가능하므로 넘김.
                if len(cats) == 1:
                    continue
                
                selected_cats = random.sample(cats, 2)
                cat_list1 = npy_dict[selected_cats[0]]
                cat_list2 = npy_dict[selected_cats[1]]
                
                indices = []
                indices.append(random.choice(cat_list1))
                indices.append(random.choice(cat_list2))
            
            else:
                indices = random.sample(range(emb_mat.shape[0]), 2)
            
                
            emb1 = emb_mat[indices[0]]
            emb2 = emb_mat[indices[1]]
            
            if flag == 0:                
                emb1 = F.normalize(emb1, p=2, dim=1)
                emb2 = F.normalize(emb2, p=2, dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=0)
            else:
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=0)
                emb1 = F.normalize(emb1, p=2, dim=0)
                emb2 = F.normalize(emb2, p=2, dim=0)
            
            d1_same.append(torch.dist(emb1, emb2, p=2))
            
            
            
            # diff spk
            indices = random.sample(spk_list, 2)
            emb_mat1 = spk_emb_dict[indices[0]] # n_utter, 10, 512
            emb_mat2 = spk_emb_dict[indices[1]] # n_utter, 10, 512
            
            emb1 = random.choice(emb_mat1)
            emb2 = random.choice(emb_mat2)
            
            if flag == 0:
                emb1 = F.normalize(emb1, p=2, dim=1)
                emb2 = F.normalize(emb2, p=2, dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=0)     
            else:
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=0)
                emb1 = F.normalize(emb1, p=2, dim=0)
                emb2 = F.normalize(emb2, p=2, dim=0)      

            d1_diff.append(torch.dist(emb1, emb2, p=2))

    fig, ax = plt.subplots()
    plt.boxplot([d1_same, d1_diff], positions=[1,0])
    ax.set_xlabel('Label')
    ax.set_ylabel('Distance')

    plt.show()

    d1 = d1_same + d1_diff
    s1 = [-1*i for i in d1]
    label_same = np.ones(len(d1_same))
    label_diff = np.zeros(len(d1_diff))
    label = list(np.concatenate([label_same, label_diff]))
    (tt1, eer1, fpr1, fnr1) = tuneThreshold.tuneThresholdfromScore(s1, label, [1, 0.1])
    
    return eer1

In [ ]:
def eer_multiple_normalize(iter=10, flag=0, n_utter=3, diff_cat_flag=False):
    
    '''
    flag(0): normalize before frame-by average
    flag(1): normalize between frame-by average and utterance-by average
    flag(2): normalize after utterance-by average
    '''
    
    assert flag == 0 or flag == 1 or flag == 2
    
    d2_same = []
    d2_diff = []
    for i in range(iter):
        for spk in spk_list:
            
            # same spk
            emb_mat = spk_emb_dict[spk] # n_utter, 10, 512
            
            # 다른 카테고리 flag
            if diff_cat_flag:
                npy_dict = spk_npy_dict[spk]
                cats = list(npy_dict.keys())
                
                # 화자의 카테고리 수가 1인 경우, 다른 카테고리에서 두개 뽑는게 불가능하므로 넘김.
                if len(cats) == 1:
                    continue
                
                selected_cats = random.sample(cats, 2)
                cat_list1 = npy_dict[selected_cats[0]]
                cat_list2 = npy_dict[selected_cats[1]]
                indices1 = random.sample(cat_list1, n_utter)
                indices2 = random.sample(cat_list2, n_utter)
            
            else:
                indices = random.sample(range(emb_mat.shape[0]), 2*n_utter)
                indices1 = indices[0:n_utter]
                indices2 = indices[n_utter:2*n_utter]
            
            emb1 = torch.index_select(emb_mat, dim=0, index=torch.tensor(indices1)) # 3, 10, 512
            emb2 = torch.index_select(emb_mat, dim=0, index=torch.tensor(indices2)) # 3, 10, 512
            
            if flag == 0:
                emb1 = F.normalize(emb1, p=2, dim=2)
                emb1 = emb1.mean(dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = F.normalize(emb2, p=2, dim=2)
                emb2 = emb2.mean(dim=1)
                emb2 = emb2.mean(dim=0)
            elif flag == 1:
                emb1 = emb1.mean(dim=1)
                emb1 = F.normalize(emb1, p=2, dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=1)
                emb2 = F.normalize(emb2, p=2, dim=1)
                emb2 = emb2.mean(dim=0)
            else:
                emb1 = emb1.mean(dim=1)
                emb1 = emb1.mean(dim=0)
                emb1 = F.normalize(emb1, p=2, dim=0)
                emb2 = emb2.mean(dim=1)
                emb2 = emb2.mean(dim=0)
                emb2 = F.normalize(emb2, p=2, dim=0)
                
            d2_same.append(torch.dist(emb1, emb2, p=2))
            
            # diff spk
            indices = random.sample(spk_list, 2)
            emb_mat1 = spk_emb_dict[indices[0]] # n_utter, 10, 512
            emb_mat2 = spk_emb_dict[indices[1]] # n_utter, 10, 512
            indices1 = random.sample(range(emb_mat1.shape[0]), n_utter)
            indices2 = random.sample(range(emb_mat2.shape[0]), n_utter)
            
            emb1 = torch.index_select(emb_mat1, dim=0, index=torch.tensor(indices1)) # 3, 10, 512
            emb2 = torch.index_select(emb_mat2, dim=0, index=torch.tensor(indices2)) # 3, 10, 512
            
            if flag == 0:
                emb1 = F.normalize(emb1, p=2, dim=2)
                emb1 = emb1.mean(dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = F.normalize(emb2, p=2, dim=2)
                emb2 = emb2.mean(dim=1)
                emb2 = emb2.mean(dim=0)
            elif flag == 1:
                emb1 = emb1.mean(dim=1)
                emb1 = F.normalize(emb1, p=2, dim=1)
                emb1 = emb1.mean(dim=0)
                emb2 = emb2.mean(dim=1)
                emb2 = F.normalize(emb2, p=2, dim=1)
                emb2 = emb2.mean(dim=0)
            else:
                emb1 = emb1.mean(dim=1)
                emb1 = emb1.mean(dim=0)
                emb1 = F.normalize(emb1, p=2, dim=0)
                emb2 = emb2.mean(dim=1)
                emb2 = emb2.mean(dim=0)
                emb2 = F.normalize(emb2, p=2, dim=0)
                
            d2_diff.append(torch.dist(emb1, emb2, p=2))
        
    fig, ax = plt.subplots()
    plt.boxplot([d2_same, d2_diff], positions=[1,0])
    ax.set_xlabel('Label')
    ax.set_ylabel('Distance')
    plt.show()


    d2 = d2_same + d2_diff
    s2 = [-1*i for i in d2]
    label_same = np.ones(len(d2_same))
    label_diff = np.zeros(len(d2_diff))
    label = list(np.concatenate([label_same, label_diff]))
    (tt2, eer2, fpr2, fnr2) = tuneThreshold.tuneThresholdfromScore(s2, label, [1, 0.1])
    
    
    return eer2

In [ ]:
# print("original method EER:", eer_original(iter=10, diff_cat_flag=False))
print("original method EER:", eer_original(iter=10, diff_cat_flag=True))

In [ ]:
# print("single method EER (normalize before average): ", eer_single_normalize(iter=10, flag=0))
# print("single method EER (normalize after average): ", eer_single_normalize(iter=10, flag=1))

print("single method EER (normalize before average): ", eer_single_normalize(iter=10, flag=0, diff_cat_flag=True))
print("single method EER (normalize after average): ", eer_single_normalize(iter=10, flag=1, diff_cat_flag=True))

In [ ]:
# print("multiple method EER (normalize before frame-by avg): ", eer_multiple_normalize(iter=10, flag=0, n_utter=3))
# print("multiple method EER (normalize between frame-by avg and utter-by avg): ", eer_multiple_normalize(iter=10, flag=1, n_utter=3))
# print("multiple method EER (normalize after utter-by avg): ", eer_multiple_normalize(iter=10, flag=2, n_utter=3))

print("multiple method EER (normalize before frame-by avg): ", eer_multiple_normalize(iter=10, flag=0, n_utter=3, diff_cat_flag=True))
print("multiple method EER (normalize between frame-by avg and utter-by avg): ", eer_multiple_normalize(iter=10, flag=1, n_utter=3, diff_cat_flag=True))
print("multiple method EER (normalize after utter-by avg): ", eer_multiple_normalize(iter=10, flag=2, n_utter=3, diff_cat_flag=True))

In [ ]:
d1_same = []
d1_diff = []

same_info = []
same_emb = []

for i in range(10):
    for spk in spk_list:

        # same spk
        emb_mat = spk_emb_dict[spk] # n_utter, 10, 512
        
        # 다른 카테고리 flag
        if True:
            npy_dict = spk_npy_dict[spk]
            cats = list(npy_dict.keys())
            
            # 화자의 카테고리 수가 1인 경우, 다른 카테고리에서 두개 뽑는게 불가능하므로 넘김.
            if len(cats) == 1:
                continue
            
            selected_cats = random.sample(cats, 2)
            cat_list1 = npy_dict[selected_cats[0]]
            cat_list2 = npy_dict[selected_cats[1]]
            
            indices = []
            indices.append(random.choice(cat_list1))
            indices.append(random.choice(cat_list2))
        
        else:
            indices = random.sample(range(emb_mat.shape[0]), 2)
        
            
        emb1 = emb_mat[indices[0]]
        emb2 = emb_mat[indices[1]]
        same_emb.append((emb1, emb2))
        
        if False:
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)
            emb1 = emb1.mean(dim=0)
            emb2 = emb2.mean(dim=0)
        else:
            emb1 = emb1.mean(dim=0)
            emb2 = emb2.mean(dim=0)
            emb1 = F.normalize(emb1, p=2, dim=0)
            emb2 = F.normalize(emb2, p=2, dim=0)
        
        d1_same.append(torch.dist(emb1, emb2, p=2))
        same_info.append((spk, indices[0], spk, indices[1]))
        
        
        # diff spk
        indices = random.sample(spk_list, 2)
        emb_mat1 = spk_emb_dict[indices[0]] # n_utter, 10, 512
        emb_mat2 = spk_emb_dict[indices[1]] # n_utter, 10, 512
        
        # emb1 = random.choice(emb_mat1)
        # emb2 = random.choice(emb_mat2)
        indices2 = []
        indices2.append(random.choice(range(emb_mat1.shape[0])))
        indices2.append(random.choice(range(emb_mat2.shape[0])))
        emb1 = emb_mat1[indices2[0]]
        emb2 = emb_mat2[indices2[1]]
        
        if False:
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)
            emb1 = emb1.mean(dim=0)
            emb2 = emb2.mean(dim=0)     
        else:
            emb1 = emb1.mean(dim=0)
            emb2 = emb2.mean(dim=0)
            emb1 = F.normalize(emb1, p=2, dim=0)
            emb2 = F.normalize(emb2, p=2, dim=0)      

        d1_diff.append(torch.dist(emb1, emb2, p=2))
        # diff_idx.append((indices[0], indices2[0], indices[1], indices2[1]))

fig, ax = plt.subplots()
plt.boxplot([d1_same, d1_diff], positions=[1,0])
ax.set_xlabel('Label')
ax.set_ylabel('Distance')

plt.show()

d1 = d1_same + d1_diff
s1 = [-1*i for i in d1]
label_same = np.ones(len(d1_same))
label_diff = np.zeros(len(d1_diff))
label = list(np.concatenate([label_same, label_diff]))
(tt1, eer1, fpr1, fnr1) = tuneThreshold.tuneThresholdfromScore(s1, label, [1, 0.1])

print(eer1)

In [ ]:
d1_same = np.array(d1_same)

Q1,Q3 = np.percentile(d1_same, [25, 75])
IQR = Q3-Q1

ul = Q3+1.5*IQR
ll = Q1-1.5*IQR
print(ul, ll, d1_same.shape, len(same_info))

outliers = np.where((d1_same > ul) | (d1_same < ll))
not_outliers = np.where((d1_same <= ul) & (d1_same >= ll))

outliers_dict = {}
for i in outliers[0]:
    info = same_info[i]
    num = outliers_dict.get(info[0], 0)
    num += 1
    outliers_dict[info[0]] = num

out_spk = list(outliers_dict.keys())
out_cnt = list(outliers_dict.values())

print(out_cnt)


In [ ]:
out_spk[10]

In [ ]:
print(same_emb[0])